In [1]:
%%bash 
#wget -N https://ikpublictutorial.blob.core.windows.net/deeplearningframeworks/tf-densenet121.tar.gz
#tar xzvf tf-densenet121.tar.gz

In [2]:
import os
import sys
import time
import multiprocessing
import numpy as np
import pandas as pd
import tensorflow as tf
from nets import densenet  # Download from https://github.com/pudae/tensorflow-densenet
from tensorflow.python.framework import dtypes
from tensorflow.python.framework.ops import convert_to_tensor
from tensorflow.contrib.data import Iterator
from tensorflow.contrib.slim.python.slim.nets import resnet_v1
from PIL import Image
import random
from common.utils import download_data_chextxray, get_imgloc_labels, get_train_valid_test_split
from common.utils import compute_roc_auc
slim = tf.contrib.slim

In [3]:
tf.__version__

'1.4.0'

In [4]:
CPU_COUNT = multiprocessing.cpu_count()
print("CPUs: ", CPU_COUNT)

CPUs:  12


In [5]:
# Globals
CLASSES = 14
WIDTH = 224
HEIGHT = 224
CHANNELS = 3
LR = 0.0001  # Effective learning-rate will decrease as BATCHSIZE rises
EPOCHS = 5
BATCHSIZE = 64  # Chainer auto scales batch
IMAGENET_RGB_MEAN = np.array([123.68, 116.78, 103.94], dtype=np.float32)
IMAGENET_SCALE_FACTOR = 0.017
TOT_PATIENT_NUMBER = 30805  # From data

In [6]:
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)
# Model checkpoint
CHKPOINT = 'tf-densenet121.ckpt'
#CHKPOINT = 'resnet_v1_50.ckpt'

chestxray/images chestxray/Data_Entry_2017.csv


In [7]:
%%time
# Download data
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 688 ms, sys: 196 ms, total: 884 ms
Wall time: 883 ms


In [8]:
#####################################################################################################
## Data Loading

In [9]:
class XrayData():
    
    def __init__(self, img_dir, lbl_file, patient_ids, mode='inference', 
                 width=WIDTH, height=HEIGHT, batch_size=BATCHSIZE, 
                 imagenet_mean=IMAGENET_RGB_MEAN, imagenet_scaling = IMAGENET_SCALE_FACTOR):
        # Get data
        self.img_locs, self.labels = get_imgloc_labels(img_dir, lbl_file, patient_ids)
        self.data_size = len(self.labels)
        self.imagenet_mean = imagenet_mean
        self.imagenet_scaling = imagenet_scaling
        self.width = width
        self.height = height
        # Create dataset
        # Performance: https://www.tensorflow.org/versions/master/performance/datasets_performance
        # Following: https://stackoverflow.com/a/48096625/6772173
        data = tf.data.Dataset.from_tensor_slices((self.img_locs, self.labels))
        # Processing
        if mode == 'training':
            print("Training")
            data = data.shuffle(self.data_size).repeat().map(self._parse_function_train,
                            num_parallel_calls=CPU_COUNT).prefetch(10*batch_size).batch(batch_size)
        else:
            print("Inference")
            data = data.map(self._parse_function_inference,
                            num_parallel_calls=CPU_COUNT).prefetch(10*batch_size).batch(batch_size)
        
        self.data = data        
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
        
        
    def _parse_function_train(self, filename, label):
        img_rgb, label = self._preprocess_image_labels(filename, label)
        # Super high CPU usuage bottlenecking GPU
        # Random crop
        img_rgb = tf.image.resize_images(img_rgb, [self.height+40, self.width+40])
        img_rgb = tf.random_crop(img_rgb, [self.height, self.width, 3])
        # Random flip
        img_rgb = tf.image.random_flip_left_right(img_rgb)
        return img_rgb, label
        
        
    def _parse_function_inference(self, filename, label):
        img_rgb, label = self._preprocess_image_labels(filename, label)
        # Resize to final dimensions
        img_rgb = tf.image.resize_images(img_rgb, [self.height, self.width])
        return img_rgb, label 
       
    
    def _preprocess_image_labels(self, filename, label):
        # load and preprocess the image
        img_decoded = tf.to_float(tf.image.decode_png(tf.read_file(filename), channels=3))
        img_centered = tf.subtract(img_decoded, self.imagenet_mean)
        img_rgb = img_centered * self.imagenet_scaling
        return img_rgb, tf.cast(label, dtype=tf.float32)

In [10]:
train_set, valid_set, test_set = get_train_valid_test_split(TOT_PATIENT_NUMBER, shuffle=True)

train:21563 valid:3080 test:6162


In [11]:
with tf.device('/cpu:0'):
    # Create dataset for iterator
    train_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=train_set,  
                             mode='training')
    valid_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=valid_set)
    test_dataset  = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=test_set)

Training
Loaded 79227 labels and 79227 images
Inference
Loaded 10930 labels and 10930 images
Inference
Loaded 21963 labels and 21963 images


In [12]:
#####################################################################################################
## Helper Functions

In [13]:
def get_symbol(model_name, in_tensor, is_training, chkpoint, out_features):
    if model_name == 'resnet50':
        # Import symbol
        with slim.arg_scope(resnet_v1.resnet_arg_scope()):
            base_model, _ = resnet_v1.resnet_v1_50(in_tensor, 
                                                   num_classes=None, 
                                                   is_training=is_training)
        # Attach extra layers
        fc = tf.layers.dense(base_model, out_features, name='output')
        # Activation function will be included in loss
        sym = tf.reshape(fc, shape=[-1, out_features])
        # Load weights for layers
        variables_to_restore = slim.get_variables_to_restore(exclude=['output'])
    elif model_name == 'densenet121':
         # Import symbol
        with slim.arg_scope(densenet.densenet_arg_scope()):
            base_model, _ = densenet.densenet121(in_tensor,
                                                 num_classes=out_features,
                                                 is_training=is_training)
            # Need to reshape from (?, 1, 1, 14) to (?, 14)
            sym = tf.reshape(base_model, shape=[-1, out_features])
        # Collect variables to restore from checkpoint
        variables_to_restore = slim.get_variables_to_restore(exclude=['densenet121/logits', 'predictions'])
    else:
        raise ValueError("Unknown model-name")
    return sym, variables_to_restore

In [14]:
def model_fn(features, labels, mode, params):
    # Create symbol
    # is_training=True?
    # https://github.com/tensorflow/models/issues/3556
    sym, variables_to_restore = get_symbol(
        model_name=params["model_name"],
        in_tensor=features, 
        #is_training=(mode == tf.estimator.ModeKeys.TRAIN),
        is_training=True,
        chkpoint=params["checkpoint"],
        out_features=params["n_classes"])
    # Predictions
    predictions = tf.sigmoid(sym)   
    # ModeKeys.PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions)
    # Optimizer & Loss
    optimizer = tf.train.AdamOptimizer(params['lr'], beta1=0.9, beta2=0.999)
    loss_fn = tf.losses.sigmoid_cross_entropy(labels, sym)
    loss = tf.reduce_mean(loss_fn)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #https://stackoverflow.com/questions/47867748/transfer-learning-with-tf-estimator-estimator-framework
        tf.train.init_from_checkpoint(params['checkpoint'], 
                              {v.name.split(':')[0]: v for v in variables_to_restore})
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))

In [15]:
def train_input_fn():
    return train_dataset.data.make_one_shot_iterator().get_next()
def valid_input_fn():
    return valid_dataset.data.make_one_shot_iterator().get_next()
def test_input_fn():
    return test_dataset.data.make_one_shot_iterator().get_next()

In [16]:
%%time
# Create Estimator
nn = tf.estimator.Estimator(model_fn=model_fn,
                            params={"lr":LR, 
                                    "checkpoint":CHKPOINT,
                                    "n_classes":CLASSES,
                                    "model_name":"densenet121"})

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2f913e10b8>, '_service': None, '_task_type': 'worker', '_num_ps_replicas': 0, '_model_dir': '/tmp/tmpos6t0f0c', '_task_id': 0, '_is_chief': True, '_save_summary_steps': 100, '_num_worker_replicas': 1, '_tf_random_seed': None, '_save_checkpoints_steps': None}
CPU times: user 4.38 ms, sys: 74 µs, total: 4.46 ms
Wall time: 4.15 ms


In [17]:
%%time
# Main training loop: 58mins 8s
# What's a good way of adding validation data here?
nn.train(train_input_fn, 
         max_steps=EPOCHS*(train_dataset.data_size//BATCHSIZE))

INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block24/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block24/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block9/x2/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block9/x2/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block16/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block16/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block7/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block7/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block

INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block8/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block8/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/transition_block3/blk/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/transition_block3/blk/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block4/x2/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block4/x2/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block16/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block16/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block2/x1/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block2/x1/Conv/weights
INFO:tensorflow:Initialize variable dense

INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block7/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block7/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block21/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block21/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block8/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block8/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block10/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block10/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block3/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block

INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block11/x2/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block11/x2/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block16/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block16/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block7/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block7/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block14/x1/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block14/x1/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block1/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block1/x1/BatchNorm/moving_variance
INF

INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block2/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block2/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block14/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block14/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block8/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block8/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block3/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block3/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block3/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block3/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize vari

...

INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block8/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block8/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block21/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block21/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block5/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block5/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block4/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block4/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block9/x2/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block9/x2/BatchNorm/moving_variance
INFO:tensorfl

INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block5/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block5/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block5/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block5/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block19/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block19/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block7/x2/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block7/x2/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block18/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block18/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable d

INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block3/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block3/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block12/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block12/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block12/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block12/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block7/x1/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block7/x1/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block8/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block8/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize vari

INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block4/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block4/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block2/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block2/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block6/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block6/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block21/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block21/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block8/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block8/x2/Conv/weights
INFO:tensorflow:Initializ

INFO:tensorflow:global_step/sec: 1.7506
INFO:tensorflow:loss = 0.1379054, step = 5401 (57.123 sec)
INFO:tensorflow:global_step/sec: 1.80447
INFO:tensorflow:loss = 0.13094689, step = 5501 (55.418 sec)
INFO:tensorflow:global_step/sec: 1.80582
INFO:tensorflow:loss = 0.13243888, step = 5601 (55.376 sec)
INFO:tensorflow:global_step/sec: 1.81221
INFO:tensorflow:loss = 0.12481674, step = 5701 (55.181 sec)
INFO:tensorflow:global_step/sec: 1.79748
INFO:tensorflow:loss = 0.1401351, step = 5801 (55.633 sec)
INFO:tensorflow:global_step/sec: 1.81898
INFO:tensorflow:loss = 0.1494477, step = 5901 (54.976 sec)
INFO:tensorflow:global_step/sec: 1.8115
INFO:tensorflow:loss = 0.15144666, step = 6001 (55.203 sec)
INFO:tensorflow:global_step/sec: 1.81354
INFO:tensorflow:loss = 0.12509559, step = 6101 (55.141 sec)
INFO:tensorflow:Saving checkpoints for 6185 into /tmp/tmpos6t0f0c/model.ckpt.
INFO:tensorflow:Loss for final step: 0.14193.
CPU times: user 7h 9min 37s, sys: 6min 59s, total: 7h 16min 37s
Wall time

In [20]:
%%time
# Main prediction loop: 2min8s
# Test AUC: 0.7904
predictions = list(nn.predict(test_input_fn))
np.array(predictions).shape
y_truth = test_dataset.labels
y_guess = np.array(predictions)
print("Test AUC: {0:.4f}".format(compute_roc_auc(y_truth, y_guess, CLASSES))) 

INFO:tensorflow:Restoring parameters from /tmp/tmpos6t0f0c/model.ckpt-6185
Full AUC [0.7875951578320867, 0.8329748067148881, 0.7521265106511058, 0.8456940354420093, 0.850031806621724, 0.8430314595626567, 0.7976936771824573, 0.9063280300412018, 0.6628289036649317, 0.7874974191692985, 0.7282501446942518, 0.7592013889107415, 0.7020246408864712, 0.809701017543283]
Test AUC: 0.7904
CPU times: user 22min 18s, sys: 24.8 s, total: 22min 43s
Wall time: 2min 8s
